In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [2]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:08<00:00, 1191.18it/s]
c:\Users\shich\Src\bottleneck_transmission\src\benchmark_utils.py:376: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_full.groupby("bin")


{'basic': {'expected_sample_burst_size': 2.998949351496268,
           'expected_sample_data_size': 16.4871204985146,
           'expected_time_to_recover': 98.37379999999999,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.1626449318822695,
             'bit_redundancy': 411.8689999999999,
             'ideal_bit_efficency': 0.19222947413368516,
             'ideal_bit_redundancy': 336.16927040212545,
             'ideal_packet_efficiency': 0.9611473706684258,
             'ideal_packet_redundancy': 67.23385408042509,
             'ideal_time_to_recover': 83.23385408042509,
             'ideal_transmitted_bits': 416.16927040212545,
             'packet_efficiency': 0.8132246594113475,
             'packet_redundancy': 82.37379999999999,
             'permeability': 0.8460977829505936,
             'saturation': 16.0,
             'transmitted_bits': 491.8689999999999}}
Found implementation: anfext_fullrank
  First time seen. Benchmarking.

benchmark: 100%|██████████| 10000/10000 [00:06<00:00, 1510.79it/s]


{'basic': {'expected_sample_burst_size': 3.0103667027696117,
           'expected_sample_data_size': 16.410413120841714,
           'expected_time_to_recover': 14.2524,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.32630861228669067,
             'bit_redundancy': 48.00859567122585,
             'ideal_bit_efficency': 0.38616746983167544,
             'ideal_bit_redundancy': 36.962450566797344,
             'ideal_packet_efficiency': 1.9308373491583772,
             'ideal_packet_redundancy': 7.392490113359468,
             'ideal_time_to_recover': 12.043170979114299,
             'ideal_transmitted_bits': 60.215854895571496,
             'packet_efficiency': 1.6315430614334536,
             'packet_redundancy': 9.601719134245169,
             'permeability': 0.8449924910270761,
             'saturation': 4.650680865754831,
             'transmitted_bits': 71.262}}
Found implementation: simple_matrix
  First time seen. Benc

...

{'basic': {'expected_sample_burst_size': 2.9896209386281587,
           'expected_sample_data_size': 16.457626353790612,
           'expected_time_to_recover': 32.921,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.15187874001397286,
             'bit_redundancy': 139.605,
             'ideal_bit_efficency': 0.17946837241400937,
             'ideal_bit_redundancy': 114.30031048773523,
             'ideal_packet_efficiency': 0.8973418620700466,
             'ideal_packet_redundancy': 22.86006209754705,
             'ideal_time_to_recover': 27.86006209754705,
             'ideal_transmitted_bits': 139.30031048773523,
             'packet_efficiency': 0.7593937000698643,
             'packet_redundancy': 27.921,
             'permeability': 0.8462702256172975,
             'saturation': 5.0,
             'transmitted_bits': 164.605}}
Found implementation: single_binary_fullrank
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3535.01it/s]


{'basic': {'expected_sample_burst_size': 2.9906289707750955,
           'expected_sample_data_size': 16.564167725540024,
           'expected_time_to_recover': 13.305399999999999,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3495333372731997,
             'bit_redundancy': 43.273595671225834,
             'ideal_bit_efficency': 0.4126409163572466,
             'ideal_bit_redundancy': 33.09923402336242,
             'ideal_packet_efficiency': 2.063204581786233,
             'ideal_packet_redundancy': 6.619846804672484,
             'ideal_time_to_recover': 11.270527670427315,
             'ideal_transmitted_bits': 56.35263835213657,
             'packet_efficiency': 1.7476666863659984,
             'packet_redundancy': 8.654719134245168,
             'permeability': 0.8470641747281041,
             'saturation': 4.650680865754831,
             'transmitted_bits': 66.52699999999999}}
Found implementation: single_binary_fullr

benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3421.58it/s]


{'basic': {'expected_sample_burst_size': 2.991153937287734,
           'expected_sample_data_size': 16.40549719611405,
           'expected_time_to_recover': 13.7782,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.33753907373639735,
             'bit_redundancy': 45.63759567122585,
             'ideal_bit_efficency': 0.399081331025267,
             'ideal_bit_redundancy': 35.01392746806681,
             'ideal_packet_efficiency': 1.995406655126335,
             'ideal_packet_redundancy': 7.002785493613361,
             'ideal_time_to_recover': 11.653466359368192,
             'ideal_transmitted_bits': 58.26733179684096,
             'packet_efficiency': 1.6876953686819869,
             'packet_redundancy': 9.12751913424517,
             'permeability': 0.8457901873516274,
             'saturation': 4.650680865754831,
             'transmitted_bits': 68.891}}
